In [1]:
import os
import json
import httpx
import requests
import weaviate

import weaviate.classes.config as wc

In [2]:
reload_db = False # 처음부터 적재 할 시 True로 바꿔 사용 
collection_name = "MiraeNVTest"
client = weaviate.connect_to_local()

if reload_db:
    client.collections.delete(collection_name)

    db = client.collections.create(name=collection_name, vectorizer_config=wc.Configure.Vectorizer.none())
else:
    try:
        db = client.collections.get(collection_name)
    except Exception:
        print(f"Collection {collection_name} not found. Please create it first.")

In [3]:
def vectorize(input_str=str) -> list[float]:
    emb_url = "https://genos.mnc.ai:3443/api/gateway/rep/serving/10/v1/embeddings"
    emb_key = "d2278640406c48b1b626ae5963fa25a1"
    emb_headers = {"Content-Type": "application/json", "Authorization": f"Bearer {emb_key}"}

    response = requests.post(url=emb_url, headers=emb_headers, json={"input": [input_str]})

    return response.json()["data"][0]["embedding"]

### 문서 적재

In [8]:
# dummy 문서를 먼저 넣어줘야 나중에 오류가 없음
with open("dummy.json", "r") as f:
    dummy = json.load(f)

question_objects = []
for chunk in dummy:
    text_vector = vectorize(chunk["text"])
    question_objects.append(
        weaviate.classes.data.DataObject(
            properties=chunk,
            vector=text_vector,
        )
    )

db.data.insert_many(question_objects)

BatchObjectReturn(_all_responses=[UUID('b43b03bd-96b0-452e-b336-d1bd46729a59')], elapsed_seconds=0.027564048767089844, errors={}, uuids={0: UUID('b43b03bd-96b0-452e-b336-d1bd46729a59')}, has_errors=False)

In [9]:
for file in os.listdir("documents"):
    if not file.endswith(".json"): continue

    print(f"Process with file {file}")
    with open(f"documents/{file}", "r") as f:
        document = json.load(f)

    question_objects = []
    for chunk in document:
        if not chunk["text"]: continue

        text_vector = vectorize(chunk["text"])
        question_objects.append(
            weaviate.classes.data.DataObject(
                properties=chunk,
                vector=text_vector
            )
        )

    if question_objects:
        db.data.insert_many(question_objects)


Process with file (별책서식2)_제17호.외국환업무현황보고서(종합금융회사).json
Process with file 230817 금융투자회사의 영업 및 업무에 관한 규정 전문.json
Process with file 8230818163249580064_html.json
Process with file 투자성향 진단 유선프로세스 추가 사항 안내_pptx.json
Process with file 영업점 대상 방문판매 FAQ안내_Ver.2_xlsx.json
Process with file 240117_AI Weekly_pptx.json
Process with file 12. 임인숙_준법관리인 선임(대행)_docx.json
Process with file 개인연급랩_portfoliobook.json
Process with file 해외주식_매수일별잔고.json
Process with file 클라우드컴퓨팅 발전 및 이용자 보호에 관한 법률 시행령(대통령령)(제33220호)(20230112).json
Process with file DAmo_DB암호화__v4.0_오류코드_pdf.json
Process with file 회의록_1.json
Process with file 직제규정(별표2-3)본사팀업무(20240507).json
Process with file 회의록_2.json
Process with file HR_MANUAL_2024.04.29_수정후_HR취합완료_추가수정요청건 (2).json
Process with file 240115_글로벌 ETF 시장 동향 240108-240114_docx.json
Process with file 전결규정(별표2)예산집행전결권한표(20240304).json
Process with file 회의록_3.json
Process with file 240213_글로벌 ETF 시장 동향 240205-240211_docx.json
Process with file 고난도금융투자상품 공지글_수정_210531_pptx.json

### Search

In [4]:
query = "STR 회의"
query_vector = vectorize(query)
search_out = db.query.hybrid(
    query=query,
    vector=query_vector, 
    limit=5)

for i, obj in enumerate(search_out.objects):
    print(f"{i}순위: ")

    print(obj.properties["major_heading"])
    print(obj.properties["text"][:30])
    print("\n")

0순위: 
회의록 STR 고도화 최종결과에 따른 변경사항 리뷰
STR 고도화 최종결과에 따른 변경사항 자료 및 반영 


1순위: 
회의록 STR 고도화 최종결과에 따른 변경사항 리뷰
3. 회의 내용 
●최종결과 확인 및 반영일정 

1)


2순위: 
회의록 STR 2차 고도화 결과 리뷰
구분,상세내용
미래에셋, •중복 적출 적응할 둘 리스트


3순위: 
회의록 STR 2차 고도화 결과 리뷰
현행 룰에 대한 2차 고도화 요건 확정 및 기존 룰의 


4순위: 
회의록 STR 2차 고도화 결과 리뷰
3. 회의 내용 
●고도화 결과 리뷰. 

1) 현행 




In [16]:
client.close()

In [7]:
del query_vector